# CPS769 - Introdução à Inteligência Artificial e Aprendizagem Generativa

Este repositório contém as listas de exercícios da disciplina CPS769 - Introdução à Inteligência Artificial e Aprendizagem Generativa, do Programa de Engenharia de Sistemas e Computação (PESC) do Instituto Alberto Luiz Coimbra de Pós-Graduação e Pesquisa de Engenharia (COPPE/UFRJ).

## LangChain

LangChain é uma estrutura para desenvolver aplicações usando modelos de linguagem grandes (LLMs). Simplifica o ciclo de vida dessas aplicações com componentes de código aberto, suporte para agentes com estados e streaming, e integração com ferramentas de monitoramento e implantação. Inclui bibliotecas como `langchain-core`, `langchain-community`, `LangGraph` e `LangServe` para criar e gerenciar aplicações complexas. Há guias e tutoriais disponíveis para ajudar na construção de chatbots, agentes e outras aplicações específicas.

Para mais detalhes, acesse [LangChain Introduction](https://python.langchain.com/v0.2/docs/introduction/).

## Objetivo

O objetivo desta atividade é se familiarizar com LangChain, seguindo o tutorial [How to return structured data from a model](https://python.langchain.com/v0.2/docs/how_to/structured_output/).

O tutorial ensina como retornar dados estruturados de um modelo utilizando o método `.with_structured_output()` em LangChain. Este método facilita a obtenção de saídas que correspondem a um esquema específico, útil para inserir dados em bancos de dados ou sistemas downstream. O tutorial explica como definir esquemas usando JSON Schema ou classes Pydantic, e como implementar exemplos práticos, como separar a introdução e o desfecho de uma piada.

## Execução do Tutorial

### Fazendo com que um modelo gere uma piada e separe a introdução da conclusão.

Intanciando a classe ChatOpenAI

In [5]:
import getpass
import os
from env import OPENAI_API_KEY # key armazenada em um arquivo que está no .gitignore para manter o sigilo

# os.environ[OPENAI_API_KEY] = getpass.getpass() # fornecido diretamente na instância llm

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini",
    openai_api_key=OPENAI_API_KEY,
)

Exemplo retornando um objeto Pydantic.

In [6]:
from typing import Optional
from langchain_core.pydantic_v1 import BaseModel, Field

class Joke(BaseModel):
    """Joke to tell user."""
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(description="How funny the joke is, from 1 to 10")


structured_llm = llm.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about cats")

Joke(setup='Why was the cat sitting on the computer?', punchline='Because it wanted to keep an eye on the mouse!', rating=7)